<a href="https://colab.research.google.com/github/Charmie13/MSCI-641-Text-Analytics_Final-Project/blob/codes/DistilBERT_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install simpletransformers

In [ ]:
!nvidia-smi

In [ ]:
import os, sys, shutil
import time
import gc
from contextlib import contextmanager
from pathlib import Path
import random
import numpy as np, pandas as pd
from tqdm import tqdm, tqdm_notebook

In [ ]:
train_set = pd.read_csv('train_reduced.csv')
test_set = pd.read_csv('test.csv')

In [ ]:
train = pd.DataFrame(columns=['text_a', 'text_b', 'labels'])
train['text_a'] = train_set['Headline']
train['text_b'] = train_set['articleBody']

In [ ]:
stances = train_set['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances
train['labels'] = stances

In [ ]:
stances = test_set['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances

In [ ]:
test = pd.DataFrame(columns=['text_a', 'text_b'])

test['text_a'] = test_set['Headline']
test['text_b'] = test_set['articleBody']

In [ ]:
test

In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
model = ClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=4, args={
    'learning_rate':2e-5,
    'num_train_epochs': 5,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'process_count': 10,
    'evaluate_during_training_steps': False,
    'train_batch_size': 4,
    'eval_batch_size': 4,
    'max_seq_length': 512,
    'save_best_model': True,
    'warmup_ratio':0.1,
    'weight_decay':0.01,
    'load_best_model_at_end':True,
    'fp16': False,
    'use_cuda': False,
    'early_stopping' : True
})

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
model.train_model(train)

  0%|          | 0/26854 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/6714 [00:00<?, ?it/s]

(33570, 0.4460102534416396)

In [ ]:
!pwd

/content


In [ ]:
test_11 = list()
for i in range(len(test)):
  test_11.append([test['text_a'][i], test['text_b'][i]])

In [ ]:
predictions, raw_outputs = model.predict(test_11)

  0%|          | 0/25413 [00:00<?, ?it/s]

  0%|          | 0/6354 [00:00<?, ?it/s]

In [ ]:
cnt = 0
for i in range(len(predictions)):
  if predictions[i] == stances[i]:
    cnt = cnt + 1
print(cnt/len(predictions))

0.7490654389485696


In [ ]:
answer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])
answer['Stance'] = predictions
answer.replace(0, 'agree',True)
answer.replace(1, 'disagree',True)
answer.replace(2, 'discuss',True)
answer.replace(3, 'unrelated',True)
answer['Headline'] = test_set['Headline']
answer['Body ID'] = test_set['Body ID']

In [ ]:
test_update = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])
test_update['Stance'] = test_set['Stance']
test_update.replace(0, 'agree',True)
test_update.replace(1, 'disagree',True)
test_update.replace(2, 'discuss',True)
test_update.replace(3, 'unrelated',True)
test_update['Headline'] = test_set['Headline']
test_update['Body ID'] = test_set['Body ID']

In [ ]:
test_update

In [ ]:
answer.to_csv('answer.csv', index=False, encoding='utf-8')

In [ ]:
!git clone https://github.com/FakeNewsChallenge/fnc-1-baseline.git

Cloning into 'fnc-1-baseline'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 23.99 KiB | 6.00 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
cd fnc-1-baseline/

/content/fnc-1-baseline


In [ ]:
!git submodule init
!git submodule update

Submodule 'fnc-1' (https://github.com/FakeNewsChallenge/fnc-1) registered for path 'fnc-1'
Cloning into '/content/fnc-1-baseline/fnc-1'...
Submodule path 'fnc-1': checked out '29d473af2d15278f0464d5e41e4cbe7eb58231f2'


In [ ]:
from utils.score import report_score,score_submission

In [ ]:
actual = test_update['Stance'].values
actual

array(['unrelated', 'unrelated', 'unrelated', ..., 'disagree', 'disagree',
       'agree'], dtype=object)

In [ ]:
df_pred = pd.read_csv('../answer.csv')
pred = df_pred['Stance'].values
print("F1 Score for DistilBERT:")
report_score(actual, pred)

F1 Score for BERT:
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    874    |    57     |    397    |    575    |
-------------------------------------------------------------
| disagree  |    216    |    88     |    122    |    271    |
-------------------------------------------------------------
|  discuss  |    598    |    99     |   2487    |   1280    |
-------------------------------------------------------------
| unrelated |    867    |    89     |   1806    |   15587   |
-------------------------------------------------------------
Score: 7718.0 out of 11651.25	(66.24181954725888%)


66.24181954725888